**S17** La antigüedad promedio de los usuarios y el nombre del usuario más antiguo cuyas última review contenga la palabra ‘pizza’

In [1]:
!pip install pyspark==3.2.1

In [2]:
from pyspark.sql import *
import datetime

In [3]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [4]:
sqlContext = SQLContext(sc)

/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [5]:
# Change the file directory to yours, the files are in a public folder in google drive:
# https://drive.google.com/drive/folders/1CsvJs0xZ9SCLtG4ci2dNGCkhqnI-1DLS?usp=drive_link
df_user = sqlContext.read.csv('/content/drive/MyDrive/2022/Orga_de_Datos/Datasets/user.csv', header=True, inferSchema=True)
df_user = df_user.dropna(subset=("user_id", "name", "yelping_since"))
rdd_user = df_user.rdd

In [6]:
DT_FORMAT = "%Y-%m-%d %H:%M:%S"

PARTE 1

**Para primera parte (promedio de antiguedades)**
1.   Convertir todas las fechas a años (floats)
2.   Sumar los años y sacar el promedio
3.   Volver a conseguir Años, Meses y Días

In [7]:
today_date = datetime.datetime.today().replace(microsecond=0)

In [8]:
total_user_antiquity = rdd_user.map(lambda x: datetime.datetime.strptime(x.yelping_since, DT_FORMAT))\
.map(lambda x: ((today_date - x), 1))\
.map(lambda x: ((x[0].days / 365) + (x[0].seconds / 31536000), 1))\
.reduce(lambda x,y: (x[0]+y[0], x[1]+y[1]))

In [9]:
mean_antiquity_years = total_user_antiquity[0]/total_user_antiquity[1]

In [10]:
years = int(mean_antiquity_years)
months = int((mean_antiquity_years - years) * 12)
days = int((mean_antiquity_years - years - (months / 12)) * 365)

In [11]:
print(f"La antiguedad promedio de los usuarios es de {years} años, {months} meses y {days} días")

La antiguedad promedio de los usuarios es de 8 años, 9 meses y 29 días


PARTE 2

**Para segunda parte (el nombre del usuario más antiguo cuya última review contenga la palabra ‘pizza’)**
1.   Reducir por key de usuario y conseguir la última review de cada uno (en reviews)
2.   Filtrar a los usuarios cuya última review tenga la palabra pizza (sacar los que no)
3.   Join con rdd de users para conseguir "yelping since"
4.   Con reduce y comparando fechas conseguir el más antiguo

In [12]:
# Change the file directory to yours, the files are in a public folder in google drive:
# https://drive.google.com/drive/folders/1CsvJs0xZ9SCLtG4ci2dNGCkhqnI-1DLS?usp=drive_link
df_review = sqlContext.read.csv('/content/drive/MyDrive/2022/Orga_de_Datos/Datasets/review.csv', header=True, inferSchema=True)
df_review = df_review.dropna(subset=("user_id", "text", "date"))
rdd_review = df_review.rdd

In [13]:
def list_is_numeric(input):

  for element in input:
    try:
        int(element)
    except ValueError:
        return False
    return True

def is_datetime(string):
  total_splitted = string.split(" ")
  if len(total_splitted) != 2: return False

  date_splitted = total_splitted[0].split("-")
  if not list_is_numeric(date_splitted): return False

  hour_splitted = total_splitted[1].split(":")
  if not list_is_numeric(hour_splitted): return False

  return len(date_splitted) == 3 and len(hour_splitted) == 3

In [14]:
rdd_review = rdd_review.filter(lambda x: is_datetime(x.date))

In [15]:
rdd_review_short = rdd_review.map(lambda x: (x.user_id, (x.date, x.text)))\
.reduceByKey(lambda x,y: (x[0], x[1]) if datetime.datetime.strptime(x[0], DT_FORMAT) > datetime.datetime.strptime(y[0], DT_FORMAT) else (y[0], y[1]))\
.filter(lambda x: "pizza" in x[1][1])\
.map(lambda x: (x[0], 1))

In [16]:
user = rdd_user.map(lambda x: (x.user_id, (x.name, x.yelping_since)))\
.join(rdd_review_short)\
.map(lambda x: (x[0], (x[1][0][0], x[1][0][1])))\
.filter(lambda x: is_datetime(x[1][1]))\
.reduce(lambda x,y: x if datetime.datetime.strptime(x[1][1], DT_FORMAT) < datetime.datetime.strptime(y[1][1], DT_FORMAT) else y)

In [17]:
print(f"El usuario más antiguo cuya última review contiene la palabra 'pizza' se llama: {user[1][0]}")

El usuario más antiguo cuya última review contiene la palabra 'pizza' se llama: mykeah
